<a href="https://colab.research.google.com/github/PetitYohan/DataMining/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projet de DataMining
Par Catherine Vanden Hende & Yohan Petit

##Initialisation du Projet

In [1]:
!wget -q --show-progress --no-check-certificate 'https://docs.google.com/uc?export=download&id=17ockZAT8Uf8fmgKPKIfKI4s2_bpCtOhd' -O kaggle.json
! pip install kaggle
! rm -r ~/.kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d razinw/dog-vs-cat
! rm -r images
! mkdir images
! unzip dog-vs-cat.zip -d images/

kaggle.json         100%[===================>]      66  --.-KB/s    in 0s      
rm: cannot remove '/root/.kaggle': No such file or directory
 99% 410M/412M [00:03<00:00, 141MB/s]
100% 412M/412M [00:03<00:00, 136MB/s]
rm: cannot remove 'images': No such file or directory
Archive:  dog-vs-cat.zip
  inflating: images/dogvscat/test/0/250.jpg  
  inflating: images/dogvscat/test/0/251.jpg  
  inflating: images/dogvscat/test/0/252.jpg  
  inflating: images/dogvscat/test/0/253.jpg  
  inflating: images/dogvscat/test/0/254.jpg  
  inflating: images/dogvscat/test/0/255.jpg  
  inflating: images/dogvscat/test/0/256.jpg  
  inflating: images/dogvscat/test/0/257.jpg  
  inflating: images/dogvscat/test/0/258.jpg  
  inflating: images/dogvscat/test/0/259.jpg  
  inflating: images/dogvscat/test/0/260.jpg  
  inflating: images/dogvscat/test/0/261.jpg  
  inflating: images/dogvscat/test/0/262.jpg  
  inflating: images/dogvscat/test/0/263.jpg  
  inflating: images/dogvscat/test/0/264.jpg  
  inflating: i

##Récupération des métadonnées dans des json

In [3]:
!pip3 install Pillow
!mkdir metadata
!mkdir metadata/train
!mkdir metadata/train/0
!mkdir metadata/train/1

from PIL import Image
from PIL.ExifTags import TAGS
import os
import json

# path to the directory
path0 = "images/dogvscat/train/0"
path1 = "images/dogvscat/train/1"

def exportMetadata(path):
  # directory
  directory = os.listdir(path)
  metadata = []
  photo = dict()

  for files in directory:
    if files.endswith ('.jpg'):
      # find file path
      file_path = os.path.join(path,files)
      # read the image data using PIL
      image = Image.open(file_path)
      # extract EXIF data
      exifdata = image.getexif()
      # iterating over all EXIF data fields
      for tag_id in exifdata:
          # get the tag name, instead of human unreadable tag id
          tag = TAGS.get(tag_id, tag_id)
          data = exifdata.get(tag_id)
          # decode bytes 
          if isinstance(data, bytes):
              data = data.decode()
          if tag != 'GPSInfo':
            photo[tag]=data
      # metadata list for each photo
      metadata.append(photo)
      photo = {}
  return metadata

# create json file
with open("metadata/train/0/metadata.json", "w") as file:
    json.dump(exportMetadata(path0), file, separators=(',', ':'), indent=4)
with open("metadata/train/1/metadata.json", "w") as file:
    json.dump(exportMetadata(path1), file, separators=(',', ':'), indent=4)

mkdir: cannot create directory ‘metadata’: File exists
mkdir: cannot create directory ‘metadata/train’: File exists
mkdir: cannot create directory ‘metadata/train/0’: File exists
mkdir: cannot create directory ‘metadata/train/1’: File exists


##Étiquetage et annotation

In [4]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans
import json

def to_tag(folder,tag):
  tags=[]
  photo = dict()
  for j in range(250):
    imgfile = Image.open(folder+str(j)+".jpg")
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = MiniBatchKMeans(n_clusters=4)
    clusters.fit(numarray)
    colors=[]
    photo["id"]=str(j)
    photo["tag"]=tag
    colors.append('#%02x%02x%02x' % (
      math.ceil(clusters.cluster_centers_[0][0]),
      math.ceil(clusters.cluster_centers_[0][1]),
      math.ceil(clusters.cluster_centers_[0][2]),
      )
    )
    photo["color"]=colors[0]
    tags.append(photo)
    photo = {}
  return tags

with open("metadata/train/0/tag.json", "w") as file:
    json.dump(to_tag("images/dogvscat/train/0/","chat"), file, separators=(',', ':'), indent=4)
with open("metadata/train/1/tag.json", "w") as file:
    json.dump(to_tag("images/dogvscat/train/1/","chien"), file, separators=(',', ':'), indent=4)

##Analyses de données

In [5]:
#Création des profiles utilisateurs
def likeUser(liked, tag1, tag2):
  photo=dict()
  likePhoto=[]
  for i in range(250):
    photo["id"]=i
    if i in liked :
      photo["like"]=1
    else : 
      photo["like"]=0
    photo["tag"]=tag1
    likePhoto.append(photo)
    photo={}
  for i in range(250):
    photo["id"]=i
    photo["like"]=0
    photo["tag"]=tag2
    likePhoto.append(photo)
    photo = {}
  return likePhoto

#User1 Chat gris
user1=[1,102,105,11,115,116,119,12,130,132,141,147,149,153,154,166,182,185,19,207,208,212,231,230,23,28,53,58,6,74,85,95]
with open("metadata/train/like_user1.json","w") as file:
  json.dump(likeUser(user1, "chat", "chien"), file, separators=(',', ':'), indent=4)
#User2 Chien roux
user2=[0,117,118,122,132,137,14,143,146,147,151,156,166,170,189,2,204,210,211,222,243,248,27,39,45,54,75]
with open("metadata/train/like_user2.json","w") as file:
  json.dump(likeUser(user2, "chien", "chat"), file, separators=(',', ':'), indent=4)

##Visualisation des données

In [23]:
import json
import operator

def countByDate(metadata, user=None):
  date = dict()
  for i in range(2):
    #if user is not None :
    with open('metadata/train/'+str(i)+'/metadata.json') as jsonFile :
      jsonObjects = json.load(jsonFile)
      jsonFile.close()
    for jsonObject in jsonObjects:
      if metadata in jsonObject:
        data = jsonObject[metadata]
        year = data.split(':')[0]
        if year in date:
          date[year]+=1
        else : 
          date[year]=1
  return date

def countByData(metadata, user=None):
  dataCount = dict()
  for i in range(2):
    #if user is not None :
    with open('metadata/train/'+str(i)+'/metadata.json') as jsonFile :
      jsonObjects = json.load(jsonFile)
      print(len(jsonObjects))
      jsonFile.close()
    for jsonObject in jsonObjects:
      if metadata in jsonObject:
        data = jsonObject[metadata]
        if data in dataCount:
          dataCount[data]+=1
        else : 
          dataCount[data]=1
  return dataCount

def countBySize(metadataWidth,metadataLength, user=None):
  dataCount = dict()
  for i in range(2):
    #if user is not None :
    with open('metadata/train/'+str(i)+'/metadata.json') as jsonFile :
      jsonObjects = json.load(jsonFile)
      jsonFile.close()
    for jsonObject in jsonObjects:
      if metadataWidth in jsonObject:
        if metadataLength in jsonObject:
          data = str(jsonObject[metadataWidth])+'x'+str(jsonObject[metadataLength])
          if data in dataCount:
            dataCount[data]+=1
          else : 
            dataCount[data]=1
  return dataCount 

print(countByDate('DateTimeOriginal'))
print(countByData('Model'))
print(countBySize('ImageWidth','ImageLength'))

max_key = max(countBySize('ImageWidth','ImageLength').items(), key=operator.itemgetter(1))[0]
print(max_key)

{'2015': 61, '2017': 76, '2013': 41, '2010': 6, '2016': 72, '2006': 2, '2018': 55, '2014': 38, '2012': 10, '2019': 32, '2011': 6, '2005': 3, '2009': 6, '2008': 2, '2007': 2}
250
250
{'Canon EOS 750D': 1, 'NIKON D3300': 7, 'Canon EOS 70D': 6, 'E-PL2': 1, 'COOLPIX L840': 1, 'Canon EOS 500D': 5, 'iPhone 7 Plus': 1, 'Canon EOS 400D DIGITAL': 3, 'ILCA-77M2': 6, 'NIKON D90': 10, 'Canon EOS 100D': 24, 'NIKON D3200': 11, 'Canon EOS 5D Mark II': 6, 'DMC-FZ7': 1, 'NIKON D5100': 13, 'NIKON D3100': 8, 'NIKON D5200': 24, 'NX300M': 7, 'Canon EOS 600D': 14, 'ILCE-7M2': 1, 'DMC-FZ48': 1, 'Canon EOS 60D': 4, 'Canon EOS 80D': 5, 'ILCE-6000': 8, 'NIKON D300S': 3, 'FinePix S100FS ': 1, 'ILCE-5000': 1, 'NIKON D750': 5, 'NIKON D3400': 2, 'NIKON D5300': 5, 'iPad': 1, 'NIKON D7100': 4, 'NIKON D7000': 3, 'SLT-A99V': 9, 'Canon EOS 200D': 6, 'NIKON D800': 2, 'NIKON D80': 3, 'DSC-P72': 1, 'DMC-FZ38': 1, 'NIKON D5000': 1, 'Canon PowerShot G3 X': 1, 'X-E2': 1, 'Canon EOS REBEL T5i': 1, 'DSC-R1': 1, 'Canon EOS REBEL